# Using ProjectPrompt Class from Gemini Generator

This notebook demonstrates how to use the `ProjectPrompt` class from the `gemini_generator.py` script to generate project documentation, including project prompts and GitHub Copilot instructions.

## Prerequisites

1.  **`gemini_generator.py`:** Ensure this file is in the parent directory relative to this notebook's location (e.g., `../gemini_generator.py`) or that the `terraform-prompt-template` directory is in your Python path. The cells below will attempt to add the parent directory to `sys.path`.
2.  **Environment Variables:**
    *   `GEMINI_API_KEY`: Your Google Gemini API key must be set as an environment variable.
3.  **Required Packages:** Make sure you have installed all necessary packages listed in `gemini_generator.py` (e.g., `google-generativeai`, `pydantic`, `pydantic-ai`, `jinja2`).
4.  **Templates:** The `gemini_generator.py` script relies on templates located in a `templates` directory (e.g., `../templates/`). Ensure this directory is accessible.


In [ ]:
import os
import sys
import json

# Add the parent directory to sys.path to find gemini_generator
# Adjust this path if your gemini_generator.py is located elsewhere
notebook_dir = os.path.dirname(os.path.abspath("__file__")) # Gets the directory of the current notebook
parent_dir = os.path.dirname(notebook_dir)
sys.path.append(parent_dir)

try:
    from gemini_generator import ProjectPrompt, ProjectOutput # Import the class
except ImportError as e:
    print(f"Error importing ProjectPrompt: {e}")
    print(f"Ensure 'gemini_generator.py' is in the correct path: {parent_dir}")
    ProjectPrompt = None # Avoid further errors if import fails

# Configure logging for the gemini_generator module if desired (optional)
import logging
logger = logging.getLogger() # Get root logger
# Clear existing handlers
for handler in logger.handlers[:]:
    logger.removeHandler(handler)
    handler.close()

logger.setLevel(logging.INFO) # Set to DEBUG for more verbose output
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(console_handler)

print(f"Attempting to import ProjectPrompt from: {os.path.join(parent_dir, 'gemini_generator.py')}")
print(f"Python sys.path includes: {parent_dir}")

In [ ]:
# Define Project Parameters
project_name = "MySampleProject"
project_prompt_text = "Create a Python application that uses a public API to fetch weather data and display it."
repo_org = "MyGitHubOrg"

# Placeholder variables (optional, defaults are provided in ProjectPrompt)
placeholder_format = "${%s}"
placeholder_vars_list = ['project_name', 'repo_org', 'project_type', 'programming_language']

# Path to the project prompt template (optional, defaults are provided in ProjectPrompt)
# This should point to 'project_prompt_template.j2' within your 'templates' directory
# Adjust the path as necessary based on your project structure.
# If gemini_generator.py is in parent_dir, and templates is in parent_dir/templates:
project_prompt_formatter_template_path = os.path.join(parent_dir, "templates", "project_prompt_template.j2")

print(f"Project Name: {project_name}")
print(f"Repo Org: {repo_org}")
print(f"Project Prompt Formatter Template Path: {project_prompt_formatter_template_path}")

In [ ]:
# Retrieve Gemini API Key
gemini_api_key = os.getenv("GEMINI_API_KEY")

if not gemini_api_key:
    print("Error: GEMINI_API_KEY environment variable not set.")
    print("Please set it before running the next cell.")
else:
    print("GEMINI_API_KEY found.")

In [ ]:
# Instantiate ProjectPrompt
project_prompt_instance = None
if ProjectPrompt and gemini_api_key:
    try:
        print("Instantiating ProjectPrompt...")
        project_prompt_instance = ProjectPrompt(
            project_name=project_name,
            project_prompt_text=project_prompt_text,
            repo_org=repo_org,
            gemini_api_key=gemini_api_key,
            main_gemini_model='gemini-1.5-flash-latest', # Using a faster model for testing
            copilot_gemini_model='gemini-1.5-flash-latest',
            token_config_overrides={"max_output_tokens": 2048}, # Example override
            enable_search_grounding=False, # Disable for faster testing if not needed
            placeholder_format=placeholder_format,
            placeholder_vars_list=placeholder_vars_list,
            project_prompt_formatter_template_path=project_prompt_formatter_template_path
        )
        print("ProjectPrompt instance created successfully.")
    except RuntimeError as e:
        print(f"RuntimeError during ProjectPrompt instantiation: {e}")
    except Exception as e:
        print(f"An unexpected error occurred during ProjectPrompt instantiation: {e}")
        import traceback
        traceback.print_exc()
else:
    if not ProjectPrompt:
        print("ProjectPrompt class not imported. Cannot instantiate.")
    if not gemini_api_key:
        print("GEMINI_API_KEY not set. Cannot instantiate ProjectPrompt.")


In [ ]:
# Access and Print Generated Content (Simple Project Prompt)
if project_prompt_instance and project_prompt_instance.simple_project_prompt_content:
    print("--- Simple Project Prompt Content ---")
    print(project_prompt_instance.simple_project_prompt_content)
else:
    print("Could not retrieve simple project prompt content.")
    if project_prompt_instance and project_prompt_instance.project_output_data and \
       "Error in Main Content Generation" in project_prompt_instance.project_output_data.readme_content:
        print("Error details from project_output_data:")
        print(project_prompt_instance.project_output_data.readme_content)

In [ ]:
# Access and Print Generated Content (Copilot Instructions)
if project_prompt_instance and project_prompt_instance.copilot_instructions_content:
    print("\n--- Copilot Instructions Content ---")
    print(project_prompt_instance.copilot_instructions_content)
else:
    print("Could not retrieve Copilot instructions content.")

In [ ]:
# Access and Print Full Project Output Data (as JSON for inspection)
if project_prompt_instance and project_prompt_instance.project_output_data:
    print("\n--- Full Project Output Data (JSON) ---")
    try:
        # Assuming ProjectOutput has a .dict() method (if it's a Pydantic model)
        # or convert to dict manually if it's a simple class
        if hasattr(project_prompt_instance.project_output_data, 'dict'):
            output_dict = project_prompt_instance.project_output_data.dict()
        else: # Fallback for non-Pydantic or objects without .dict()
            output_dict = project_prompt_instance.project_output_data.__dict__
        print(json.dumps(output_dict, indent=2))
    except Exception as e:
        print(f"Could not serialize project_output_data to JSON: {e}")
        print("Raw project_output_data:", project_prompt_instance.project_output_data)
else:
    print("Could not retrieve full project output data.")